# Purpose

**2022-08-15: v0.6.0**
<br>Test `dask.delayed` to run aggregation of multiple subreddits in parallel.
With the new project we expect to aggregate posts for over 300k subreddits. For most of the process, each subreddit can be processed independently of other subreddits, so it makes sense to try to split up the work so we can speed things up.

**2022-06-29: v0.5.0**
<br>Because we embedded post & text as a single embedding and we didn't use MLflow to create those embeddings, it's easier to  run the embeddings in this notebook rather than to re-use or re-write the old `AggregateEmbeddings` class.

Provenance:
* `v0.4.1 / djb_03.01-2021-12-aggregate_v041_posts_and_comments_pandas.ipynb`

# Notebook setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
from datetime import datetime, timedelta
import gc
import os
import logging
from logging import info
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import seaborn as sns

import dask
from dask import dataframe as dd
from tqdm.auto import tqdm

import mlflow
import hydra

import subclu
from subclu.utils.hydra_config_loader import LoadHydraConfig
from subclu.models.aggregate_embeddings import (
    AggregateEmbeddings, AggregateEmbeddingsConfig,
    load_config_agg_jupyter, get_dask_df_shape,
)
from subclu.models import aggregate_embeddings_pd

from subclu.utils import set_working_directory, get_project_subfolder
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric,
    elapsed_time,
)
from subclu.utils.mlflow_logger import MlflowLogger, save_pd_df_to_parquet_in_chunks
from subclu.eda.aggregates import (
    compare_raw_v_weighted_language
)
from subclu.utils.data_irl_style import (
    get_colormap, theme_dirl
)

from subclu.i18n_topic_model_batch.subclu2.utils.data_loaders_gcs import LoadSubredditsGCS


print_lib_versions([dask, hydra, mlflow, np, pd, plotly, sns, subclu])

python		v 3.7.10
===
dask		v: 2021.06.0
hydra		v: 1.1.0
mlflow		v: 1.16.0
numpy		v: 1.19.5
pandas		v: 1.2.4
plotly		v: 4.14.3
seaborn		v: 0.11.1
subclu		v: 0.6.0


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Set Local model paths

In [4]:
manual_model_timestamp = datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')
path_this_model = get_project_subfolder(
    f"data/models/aggregate_embeddings/manual_v060_{manual_model_timestamp}"
)
Path.mkdir(path_this_model, parents=True, exist_ok=True)
path_this_model

PosixPath('/home/jupyter/subreddit_clustering_i18n/data/models/aggregate_embeddings/manual_v060_2022-08-16_084129')

# Load config for embeddings aggregation

For v0.6.0 embeddings I didn't use mlflow to track the embeddings inference. We'll need to get them from these folders in GCS:

- [Subreddit metadata](https://console.cloud.google.com/storage/browser/i18n-subreddit-clustering/i18n_topic_model_batch/runs/20220629/subreddits/text/embedding/2022-06-29_084555)
    - `i18n-subreddit-clustering/i18n_topic_model_batch/runs/20220629/subreddits/text/embedding/2022-06-29_084555`
- [Post + Comment Text (already combined)](https://console.cloud.google.com/storage/browser/i18n-subreddit-clustering/i18n_topic_model_batch/runs/20220629/post_and_comment_text_combined/text_subreddit_seeds/embedding/2022-06-29_091925)
    - `i18n-subreddit-clustering/i18n_topic_model_batch/runs/20220629/post_and_comment_text_combined/text_subreddit_seeds/embedding/2022-06-29_091925`



In [5]:
cfg_agg_embeddings = LoadHydraConfig(
    config_name='aggregate_embeddings_v0.6.0.yaml',
    config_path="../config",
    overrides=[
        f"agg_style=serial",
    ],
)
print(cfg_agg_embeddings.config_dict.keys())

dict_keys(['data_text_and_metadata', 'data_embeddings_to_aggregate', 'aggregate_params', 'bucket_output', 'mlflow_tracking_uri', 'mlflow_experiment', 'n_sample_subreddits', 'n_sample_posts_files', 'n_sample_comments_files', 'agg_style'])


In [6]:
# cfg_agg_embeddings.config_dict['data_embeddings_to_aggregate']

In [7]:
for k_, v_ in cfg_agg_embeddings.config_dict.items():
    if isinstance(v_, dict):
        print(f"{k_}:")
        for k2_, v2_ in v_.items():
            print(f"    {k2_}: {v2_}")
    else:
        print(f"{k_}: {v_}")

data_text_and_metadata:
    dataset_name: v0.6.0 inputs. ~110k seed subreddits, ~340k with 3+ posts, ~700k total subreddits
    bucket_name: i18n-subreddit-clustering
    folder_subreddits_text_and_meta: i18n_topic_model_batch/runs/20220811/subreddits/text
    folder_posts_text_and_meta: i18n_topic_model_batch/runs/20220811/posts
    folder_comments_text_and_meta: i18n_topic_model_batch/runs/20220811/comments
    folder_post_and_comment_text_and_meta: i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all
data_embeddings_to_aggregate:
    bucket_embeddings: i18n-subreddit-clustering
    post_and_comments_folder_embeddings: i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all/embedding/2022-08-11_084218
    subreddit_desc_folder_embeddings: i18n_topic_model_batch/runs/20220811/subreddits/text/embedding/2022-08-11_082859
    col_subreddit_id: subreddit_id
aggregate_params:
    min_post_and_comment_text_len: 3
    agg_post_post_and_comment_wei

# Start MLflow & Log base params

In [8]:
mlf = MlflowLogger(tracking_uri=cfg_agg_embeddings.config_dict['mlflow_tracking_uri'])

In [9]:
mlflow_experiment = cfg_agg_embeddings.config_dict['mlflow_experiment']
# 'v0.6.0_mUSE_aggregates', 'v0.6.0_mUSE_aggregates_test'


t_start_agg_embed = datetime.utcnow()
info(f"== Start run_aggregation() method ==")


info(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")
mlf.set_experiment(mlflow_experiment)
mlflow.start_run()
mlf.add_git_hash_to_active_run()
mlf.set_tag_hostname(key='host_name')
mlf.log_param_hostname(key='host_name')
mlf.log_cpu_count()
mlf.log_ram_stats(param=True, only_memory_used=False)

08:41:31 | INFO | "== Start run_aggregation() method =="
08:41:31 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-100-2021-04-28-djb-eda-german-subs/mlruns.db"
08:41:31 | INFO | "host_name: djb-100-2021-04-28-djb-eda-german-subs"
08:41:31 | INFO | "cpu_count: 96"
08:41:31 | INFO | "RAM stats:
{'memory_used_percent': '7.51%', 'memory_total': '1,444,961', 'memory_used': '108,493', 'memory_free': '1,175,788'}"


{'memory_total': 1444961,
 'memory_used_percent': 0.0750836873797978,
 'memory_used': 108493,
 'memory_free': 1175788}

In [10]:
# set weights
# Normalize them by dividing by 100
WEIGHT_POST_COMMENT = (
    cfg_agg_embeddings.config_dict['aggregate_params']['agg_post_post_and_comment_weight'] / 100
)
WEIGHT_SUB_META = (
    cfg_agg_embeddings.config_dict['aggregate_params']['agg_post_subreddit_desc_weight'] / 100
)
print(WEIGHT_POST_COMMENT + WEIGHT_SUB_META)
assert(1.0 == WEIGHT_POST_COMMENT + WEIGHT_SUB_META)


gcs_sub_embeddings = cfg_agg_embeddings.config_dict['data_embeddings_to_aggregate']['subreddit_desc_folder_embeddings']
print(gcs_sub_embeddings)
gcs_post_comment_embeddings = cfg_agg_embeddings.config_dict['data_embeddings_to_aggregate']['post_and_comments_folder_embeddings']
print(gcs_post_comment_embeddings)

mlflow.log_params(
    {
        'embeddings_bucket': cfg_agg_embeddings.config_dict['data_embeddings_to_aggregate']['bucket_embeddings'],
        'embeddings_subreddit_path': gcs_sub_embeddings,
        'embeddings_post_and_comments_path': gcs_post_comment_embeddings,
        'weight_post_and_comments': WEIGHT_POST_COMMENT,
        'weight_subreddit_meta': WEIGHT_SUB_META,
    }
)
for k_, v_ in cfg_agg_embeddings.config_dict.items():
    if isinstance(v_, str):
        try:
            mlflow.log_param(k_, v_)
        except Exception as e:
            print(e)

1.0
i18n_topic_model_batch/runs/20220811/subreddits/text/embedding/2022-08-11_082859
i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all/embedding/2022-08-11_084218


# Load data

In [11]:
%%time
t_start_data_load_ = datetime.utcnow()

subs_v = LoadSubredditsGCS(
    bucket_name=cfg_agg_embeddings.config_dict['data_embeddings_to_aggregate']['bucket_embeddings'],
    gcs_path=gcs_sub_embeddings,
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=None,
    col_unique_check='subreddit_id',
    df_format='pandas',
    unique_check=True,
    verbose= True,
    
    n_sample_files=None,
    n_files_slice_start=None,
    n_files_slice_end=None,
)
subs_v.local_cache()

df_v_subs = subs_v.read_as_one_df()
r_subs, c_subs = df_v_subs.shape
mlflow.log_metrics(
    {
        f"df_v_subs-rows": r_subs,
        f"df_v_subs-cols": c_subs,
    }
)
print(f"{r_subs:,.0f} rows, {c_subs:,.0f} cols")

08:41:32 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/i18n-subreddit-clustering/i18n_topic_model_batch/runs/20220811/subreddits/text/embedding/2022-08-11_082859"
08:41:32 | INFO | "  7 <- Files matching prefix"
08:41:32 | INFO | "  7 <- Files to check"
08:41:32 | INFO | "    000000000000-131971_by_514.parquet <- File already exists, not downloading"
08:41:32 | INFO | "    000000000001-198630_by_514.parquet <- File already exists, not downloading"
08:41:32 | INFO | "    000000000002-441159_by_514.parquet <- File already exists, not downloading"
08:41:32 | INFO | "    2022-08-11_08-28-59_vectorize_text.log <- File already exists, not downloading"
08:41:32 | INFO | "  Files already cached: 4"
08:41:32 | INFO | "  Files already downloaded."
08:41:32 | INFO | "  df format: pandas"
08:41:37 | INFO | "  Checking ID uniqueness..."


771,760 rows, 514 cols
CPU times: user 4.47 s, sys: 4.3 s, total: 8.78 s
Wall time: 6.19 s


In [12]:
# gsutil is usually faster than the python library.
remote_bucket_and_key = 'i18n-subreddit-clustering/i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all/embedding/2022-08-11_084218'
remote_gs_path = f'gs://{remote_bucket_and_key}'

# Need to remove the last part of the local path otherwise we'll get duplicate subfolders:
#. top/2021-12-14/2021-12-14 instead of top/2021-12-14
local_f = f"/home/jupyter/subreddit_clustering_i18n/data/local_cache/{'/'.join(remote_bucket_and_key.split('/')[:-1])}"
Path(local_f).mkdir(parents=True, exist_ok=True)

# print(f"Remote path:\n  {remote_gs_path}")
# print(f"Local path:\n  {local_f}")

print(
    f"gsutil -m cp -r -n {remote_gs_path} {local_f}"
)
# gsutil -m cp -r -n $remote_gs_path $local_f

gsutil -m cp -r -n gs://i18n-subreddit-clustering/i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all/embedding/2022-08-11_084218 /home/jupyter/subreddit_clustering_i18n/data/local_cache/i18n-subreddit-clustering/i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all/embedding


In [13]:
%%time

pc_v = LoadSubredditsGCS(
    bucket_name=cfg_agg_embeddings.config_dict['data_embeddings_to_aggregate']['bucket_embeddings'],
    gcs_path=gcs_post_comment_embeddings,
    local_cache_path="/home/jupyter/subreddit_clustering_i18n/data/local_cache/",
    columns=None,
    col_unique_check='post_id',
    df_format='pandas',
    unique_check=False,
    verbose= True,
    
    n_sample_files=cfg_agg_embeddings.config_dict['n_sample_posts_files'],  # None,
    n_files_slice_start=None,  # None,
    n_files_slice_end=None,  # None, 
)
pc_v.local_cache()

df_v_pc = pc_v.read_as_one_df()
r_pc, c_pc = df_v_pc.shape
mlflow.log_metrics(
    {
        f"df_v_post_comments-rows": r_pc,
        f"df_v_post_comments-cols": c_pc,
    }
)
print(f"{r_pc:,.0f} rows, {c_pc:,.0f} cols")

t_data_load = elapsed_time(start_time=t_start_data_load_, log_label='Data Loading Time', verbose=True)
mlflow.log_metric('time_fxn-data_loading_time',
                  t_data_load / timedelta(minutes=1)
                  )
mlf.log_ram_stats(only_memory_used=True)

08:41:39 | INFO | "  Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/i18n-subreddit-clustering/i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all/embedding/2022-08-11_084218"
08:41:39 | INFO | "  197 <- Files matching prefix"
08:41:39 | INFO | "  197 <- Files to check"
08:41:39 | INFO | "    000000000000-264431_by_515.parquet <- File already exists, not downloading"
08:41:39 | INFO | "    000000000001-249532_by_515.parquet <- File already exists, not downloading"
08:41:39 | INFO | "    000000000002-308094_by_515.parquet <- File already exists, not downloading"
08:41:39 | INFO | "    000000000003-331082_by_515.parquet <- File already exists, not downloading"
08:41:39 | INFO | "    000000000004-356401_by_515.parquet <- File already exists, not downloading"
08:41:39 | INFO | "    000000000005-331679_by_515.parquet <- File already exists, not downloading"
08:41:39 | INFO | "    000000000006-253861_by_515.parquet <- 

51,906,348 rows, 515 cols


08:45:18 | INFO | "RAM stats:
{'memory_used_percent': '30.61%', 'memory_used': '442,282'}"


CPU times: user 6min 48s, sys: 16min 57s, total: 23min 46s
Wall time: 3min 40s


{'memory_used_percent': 0.3060857697889424, 'memory_used': 442282}

# Set weights & create copy dfs for new weights

In [14]:
l_ix_sub_level = ['subreddit_id', 'subreddit_name']
l_ix_post_level = l_ix_sub_level + ['post_id']

l_embedding_cols = [c for c in df_v_pc if c.startswith('embeddings_')]
print(len(l_embedding_cols))

512


In [15]:
%%time
df_v_pc_weighted = df_v_pc.copy()

df_v_subs_weighted = df_v_subs.copy()

# should be True b/c they're copies
print(np.allclose(df_v_pc_weighted.iloc[:1000,3:515], df_v_pc.iloc[:1000,3:515]))
print(np.allclose(df_v_subs_weighted.iloc[:1000,2:515], df_v_subs.iloc[:1000,2:515]))

# apply weight to all posts & subreddit meta at once (vectorized)
info(f"Initializing weighted SUBS meta")
df_v_subs_weighted[l_embedding_cols] = df_v_subs_weighted[l_embedding_cols] * WEIGHT_SUB_META

info(f"Initializing weighted POSTS embeddings")
df_v_pc_weighted[l_embedding_cols] = df_v_pc_weighted[l_embedding_cols] * WEIGHT_POST_COMMENT

# NOW they shouldn't be equal (Should be False)
print(np.allclose(df_v_pc_weighted.iloc[:1000,3:515], df_v_pc.iloc[:1000,3:515]))
print(np.allclose(df_v_subs_weighted.iloc[:1000,2:515], df_v_subs.iloc[:1000,2:515]))

08:46:06 | INFO | "Initializing weighted SUBS meta"


True
True


08:46:08 | INFO | "Initializing weighted POSTS embeddings"


False
False
CPU times: user 1min 40s, sys: 1min 45s, total: 3min 25s
Wall time: 3min 25s


In [16]:
# counts_describe(df_v_pc[l_ix_post_level])

# Aggregate to Post-Level: Post&Comments + Subreddit Meta

It's better to let pandas handle the interations with `.groupby('subreddit_id')`. Otherwise we have to create masks for each subreddit that can take much longer (17+ hours).

- ETA with masks: +17.6 hours
- ETA with groupby ~2.5 hours

```
# mask:
0%  329/81973 [04:18<17:42:36, 1.28it/s]

# .groupby()
6% 4751/81973 [09:56<2:35:06, 8.30it/s]
```

---

Updates using `dask.delayed`:
By combining .groupby() + `dask.delayed` we can process things ~3x faster:


```
# .groupby(), no dask delayed
100% 3467/3467 [08:20<00:00, 6.97it/s]
  0:08:21.661816 <- Total Agg fxn time time elapsed


# .groupby() + dask.delayed(....to_numpy())
100% 3467/3467 [02:31<00:00, 23.08it/s]
  Wall time: 2min 38s


# masks with dask.delayed():
#  This is 2x faster than serial processing, but .groupby() + dask.delayed() is much faster
100% 3467/3467 [00:11<00:00, 299.85it/s]
05:44:20 | INFO | "Define new C1 df DAG in dask"
05:44:20 | INFO | "COMPUTE new C1 df START"
05:48:20 | INFO | "COMPUTE new C1 df DONE"
05:48:20 | INFO | "  0:04:11.393036 <- Total Agg fxn time time elapsed"
CPU times: user 4min 33s, sys: 24.3 s, total: 4min 57s
Wall time: 4min 12s


```


In [19]:
%%time
# set style so that we can try output & time in either format
AGG_STYLE = cfg_agg_embeddings.config_dict['agg_style']  # serial v. dask.delayed

info(f"Start C1 - posts + comments + sub descriptions with format: `{AGG_STYLE}`")
t_start_agg_post_c1 = datetime.utcnow()

l_df_c1_weights = list()

if AGG_STYLE == 'serial':
    for s_id, df_ in tqdm(
        df_v_pc_weighted.groupby('subreddit_id'),
        ascii=True, mininterval=5,
    ):
        df_.loc[:, l_embedding_cols] = np.add(
            df_v_subs_weighted[df_v_subs_weighted['subreddit_id'] == s_id][l_embedding_cols].to_numpy(),
            df_[l_embedding_cols]
        )
        l_df_c1_weights.append(df_)
        del df_

    info(f"Create new C1 df")
    df_posts_agg_c1 = pd.concat(l_df_c1_weights, ignore_index=True)

elif AGG_STYLE == 'dask_delayed':
    for s_id, df_ in tqdm(
        df_v_pc_weighted.groupby('subreddit_id'),
        ascii=True, mininterval=5,
    ):
        df_pc_embeddings_ = dask.delayed(np.add)(
            # df_v_subs_weighted[df_v_subs_weighted['subreddit_id'] == s_id][l_embedding_cols].to_numpy(),
            dask.delayed(df_v_subs_weighted[df_v_subs_weighted['subreddit_id'] == s_id][l_embedding_cols].to_numpy()),
            dask.delayed(df_[l_embedding_cols])
        )
        l_df_c1_weights.append(
            dask.delayed(pd.concat)([dask.delayed(df_[l_ix_post_level]), df_pc_embeddings_], ignore_index=False, axis=1)
        )

    info(f"Define new posts C1 df DAG in dask")
    df_posts_agg_c1_delayed = dask.delayed(pd.concat)(l_df_c1_weights, ignore_index=True)

    info(f"COMPUTE new C1 df START")
    df_posts_agg_c1 = df_posts_agg_c1_delayed.compute()
    info(f"COMPUTE new C1 df DONE")
    
else:
    raise NotImplementedError(f'Other agg style not implemented: {AGG_STYLE}')

    
r_, c_ = df_posts_agg_c1.shape
mlflow.log_metrics(
    {
        f"df_posts_agg_c1-rows": r_,
        f"df_posts_agg_c1-cols": c_,
    }
)
print(f"{r_:,.0f} rows, {c_:,.0f} cols")
del r_, c_

t_agg_pc_c1 = elapsed_time(start_time=t_start_agg_post_c1, log_label='Total Agg fxn time', verbose=True)
mlflow.log_metric('time_fxn-df_posts_agg_c1_no_delay',
                  t_agg_pc_c1 / timedelta(minutes=1)
                  )
info(f"C1 - post level complete")

08:50:55 | INFO | "Start C1 - posts + comments + sub descriptions with format: `serial`"


  0%|          | 0/705963 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
df_posts_agg_c1.info()

NameError: name 'df_posts_agg_c1' is not defined

In [ ]:
df_posts_agg_c1.iloc[:5, :10]

### Save post-level

In [ ]:
d_dfs_to_save = defaultdict(dict)

In [ ]:
%%time
d_dfs_to_save['df_posts_agg_c1']['local'] = (
    path_this_model / f"df_posts_agg_c1_{datetime.utcnow().strftime('%Y-%m-%d_%H%M')}"
)

save_pd_df_to_parquet_in_chunks(
    df_posts_agg_c1,
    d_dfs_to_save['df_posts_agg_c1']['local'],
    write_index=False
)

info(f"  Logging df to mlflow...")
mlflow.log_artifacts(d_dfs_to_save['df_posts_agg_c1']['local'], artifact_path='df_posts_agg_c1')

# Aggregate to Subreddit Level

In [ ]:
df_posts_for_embeddings[df_posts_for_embeddings[c_post_embedding_count] >= 1][c_post_embedding_count].describe()

In [ ]:
df_posts_for_embeddings.head()

In [ ]:
df_posts_for_embeddings.tail()

In [ ]:
df_v_subs.head()

In [ ]:
%%time

# first, figure out how many posts each subreddit has
info(f"Count posts per subreddit...")
c_post_embedding_count = 'posts_for_embeddings_count'


df_posts_for_embedding_count = (
    df_posts_agg_c1
    .groupby(l_ix_sub_level, as_index=False)
    .agg(**{c_post_embedding_count: ('post_id', 'nunique')})
)
# fill subs that have no posts
df_posts_for_embedding_count = pd.concat(
    [
        df_posts_for_embedding_count, 
        df_v_subs[
            ~df_v_subs['subreddit_id'].isin(df_posts_agg_c1['subreddit_id'])
        ][l_ix_sub_level].assign(**{c_post_embedding_count: 0})
    ],
    axis=0
)

# min_posts >= -> regular mean. If it's less than this, then mix in subreddit_description into average
n_min_posts_for_regular_mean = 3
subreddits_above_n_ = (
    df_posts_for_embedding_count
    [df_posts_for_embedding_count[c_post_embedding_count] >= n_min_posts_for_regular_mean]
    ['subreddit_id']
)
subreddits_below_n_ = set(df_v_subs['subreddit_id']) - set(subreddits_above_n_)
mask_min_posts_for_reg_mean = df_posts_agg_c1['subreddit_id'].isin(subreddits_above_n_)


info(f"SUBREDDIT-LEVEL C1 - posts + comments + sub descriptions")
t_start_agg_subs_c1 = datetime.utcnow()

# 3+ posts: simple mean()
info(f"Mean for subs above threshold: {n_min_posts_for_regular_mean}")
df_subs_agg_c1_Nplus = (
    df_posts_agg_c1[mask_min_posts_for_reg_mean]
    .groupby(l_ix_sub_level, as_index=False)
    .mean()
)

# calculate mean for all other subs: add UNWEIGHTED subreddit_description into averages
info(f"Calculating mean for subs BELOW post threshold...")
df_subs_agg_c1_Nbelow = (
    pd.concat(
        [
            df_posts_agg_c1[~mask_min_posts_for_reg_mean],
            df_v_subs[df_v_subs['subreddit_id'].isin(subreddits_below_n_)]
        ]
    )
    .groupby(l_ix_sub_level, as_index=False)
    .mean()
)
info(f"Combining all subreddits...")
df_subs_agg_c1 = (
    df_posts_for_embedding_count
    .merge(
        pd.concat([df_subs_agg_c1_Nplus, df_subs_agg_c1_Nbelow]),
        how='outer',
        on=l_ix_sub_level
    )
    .sort_values(by=l_ix_sub_level)
)

# Check for dupes
assert(len(df_subs_agg_c1) == df_subs_agg_c1['subreddit_id'].nunique()), f"Found duplicate subreddit_ids"

r_, c_ = df_subs_agg_c1.shape
mlflow.log_metrics(
    {
        f"df_subs_agg_c1-rows": r_,
        f"df_subs_agg_c1-cols": c_,
    }
)
print(f"{r_:,.0f} rows, {c_:,.0f} cols")
del r_, c_

t_agg_subs_c1 = elapsed_time(start_time=t_start_agg_subs_c1, log_label='Total Agg fxn time', verbose=True)
mlflow.log_metric('time_fxn-df_subs_agg_c1',
                  t_agg_subs_c1 / timedelta(minutes=1)
                  )
info(f"  <- df_subs_agg_c1.shape (posts + comments + sub description)")
mlf.log_ram_stats(only_memory_used=True)

In [ ]:
df_subs_agg_c1.iloc[-8:, :10]

In [ ]:
df_subs_agg_c1.iloc[10:18, :10]

### Save Subreddit level

This one we can save as a pandas df, no need to split it into multiple files

In [ ]:
# %%time
# df_subs_agg_c1.to_parquet(
#     path_this_model / f"df_subs_agg_c1-{datetime.utcnow().strftime('%Y-%m-%d_%H%M')}.parquet"
# )

In [ ]:
%%time
d_dfs_to_save['df_subs_agg_c1']['local'] = (
    path_this_model / f"df_subs_agg_c1-{datetime.utcnow().strftime('%Y-%m-%d_%H%M')}"
)

save_pd_df_to_parquet_in_chunks(
    df_subs_agg_c1,
    d_dfs_to_save['df_subs_agg_c1']['local'],
    write_index=False
)

info(f"  Logging df to mlflow...")
mlflow.log_artifacts(d_dfs_to_save['df_subs_agg_c1']['local'], artifact_path='df_subs_agg_c1')

In [ ]:
# finish logging total time + end mlflow run
total_fxn_time = elapsed_time(start_time=t_start_agg_embed, log_label='Total Agg fxn time', verbose=True)
mlflow.log_metric('time_fxn-full_aggregation_fxn_minutes',
                  total_fxn_time / timedelta(minutes=1)
                  )
mlflow.end_run()

In [21]:
mlflow.end_run("FAILED")